In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from tqdm.notebook import tqdm
import numpy as np
import texar.torch as tx
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from lazy_dataset import LazyDataset
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
import pytorch_warmup as warmup

In [2]:
import wandb
wandb.init(project='ProjectX_transformer', name="transformer")

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: holmeswww (use `wandb login --relogin` to force relogin)


In [3]:
torch.manual_seed(10708)
np.random.seed(seed=10708)

In [4]:
order = ['GSOC', 'CT2019B_XCO2_global_7', 'Tarnocai', 'ERA5', 'CT2019B_XCO2_global_10', 'CT2019B_flux_fuel', 'CT2019B_XCO2_global_6', 'CT2019B_XCO2_global_8', 'CT2019B_XCO2_global_9', 'CWFIS', 'CT2019B_flux_fire', 'VIIRS', 'CT2019B_XCO2_global_4', 'MODIS', 'CT2019B_XCO2_global_3', 'CT2019B_XCO2_global_2', 'CT2019B_XCO2_global_0', 'CT2019B_XCO2_global_5', 'CT2019B_XCO2_global_1']


In [5]:
dataset = LazyDataset(order = order)
idx = TensorDataset(torch.from_numpy(np.arange(len(dataset))))
lengths = [int(len(idx)*0.8), int(len(idx)*0.02), len(idx) - int(len(idx)*0.8) - int(len(idx)*0.02)]
train, val, test = torch.utils.data.dataset.random_split(idx, lengths)
train_loader = DataLoader(train, batch_size=1, shuffle=True)
test_loader = DataLoader(test, batch_size=1, shuffle=False)
validation_loader = DataLoader(val, batch_size=1, shuffle=False)

GSOC ['carbon']
CT2019B_XCO2_global_7 ['h', 'e', 'i', 'g', 'h', 't', '_', '7']
Tarnocai ['PEATLAND_P', 'TOCC', 'BOG_PCT', 'FEN_PCT', 'SWAMP_PCT', 'MARSH_PCT']
ERA5 ['t2m', 'swvl1', 'swvl2', 'swvl3', 'swvl4', 'stl1', 'stl2', 'stl3', 'stl4', 'lai_lv', 'lai_hv', 'tp', 'u10', 'v10']
CT2019B_XCO2_global_10 ['h', 'e', 'i', 'g', 'h', 't', '_', '1', '0']
CT2019B_flux_fuel ['flux_fuel']
CT2019B_XCO2_global_6 ['h', 'e', 'i', 'g', 'h', 't', '_', '6']
CT2019B_XCO2_global_8 ['h', 'e', 'i', 'g', 'h', 't', '_', '8']
CT2019B_XCO2_global_9 ['h', 'e', 'i', 'g', 'h', 't', '_', '9']
CWFIS ['burned_pct_cat', 'burned_cause', 'fire_size_agency', 'fire_size_calc']
CT2019B_flux_fire ['flux_fire']


/mnt/Heavy_Duty/deep/peat/lazy_dataset.py:30: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  data.append((hf["data"], hf["dates"].value.view('<M8[D]'), feats, True))


VIIRS ['bright_ti4', 'confidence', 'frp']
CT2019B_XCO2_global_4 ['h', 'e', 'i', 'g', 'h', 't', '_', '4']
MODIS ['brightness', 'confidence', 'frp']
CT2019B_XCO2_global_3 ['h', 'e', 'i', 'g', 'h', 't', '_', '3']
CT2019B_XCO2_global_2 ['h', 'e', 'i', 'g', 'h', 't', '_', '2']
CT2019B_XCO2_global_0 ['h', 'e', 'i', 'g', 'h', 't', '_', '0']
CT2019B_XCO2_global_5 ['h', 'e', 'i', 'g', 'h', 't', '_', '5']
CT2019B_XCO2_global_1 ['h', 'e', 'i', 'g', 'h', 't', '_', '1']


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [6]:
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(out_channels)
        )
        self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=2),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        return F.relu_(self.double_conv(x) + self.downsample(x))

class ConvEnc(nn.Module):
    
    def __init__(self, in_channels, hidden_dim = 256):
        super().__init__()
        self.first = nn.Sequential(nn.Conv2d(in_channels, hidden_dim, kernel_size=5, padding=2), nn.BatchNorm2d(hidden_dim), nn.ReLU(inplace=True))
        self.L = nn.ModuleList([
            DoubleConv(hidden_dim, hidden_dim),
            DoubleConv(hidden_dim, hidden_dim),
            DoubleConv(hidden_dim, hidden_dim)
        ])

    def forward(self, x):
        x=self.first(x)
        identity = x
        for i in range(len(self.L)):
            x = self.L[i](x)
        return x

class ResBlockGenerator(nn.Module):

    def __init__(self, in_channels, out_channels):
        super(ResBlockGenerator, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, 1, padding=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, 1, padding=1)
        nn.init.kaiming_uniform_(self.conv1.weight.data, mode='fan_in', nonlinearity='relu')
        nn.init.kaiming_uniform_(self.conv2.weight.data, mode='fan_in', nonlinearity='relu')

        self.model = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(),
            nn.Upsample(scale_factor=2),
            self.conv1,
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            self.conv2
            )
        self.bypass = nn.Upsample(scale_factor=2)

    def forward(self, x):
        return self.model(x) + self.bypass(x)

class ConvDec(nn.Module):
    
    def __init__(self, out_channels, hidden_dim = 256):
        super().__init__()
        self.final = nn.Conv2d(hidden_dim, out_channels, 3, stride=1)
        nn.init.xavier_uniform(self.final.weight.data, nn.init.calculate_gain('sigmoid'))
#         self.first = nn.Sequential(nn.Conv2d(in_channels, hidden_dim, kernel_size=5, padding=2), nn.BatchNorm2d(hidden_dim), nn.ReLU(inplace=True))
        self.model = nn.Sequential(
            ResBlockGenerator(hidden_dim, hidden_dim),
            ResBlockGenerator(hidden_dim, hidden_dim),
            ResBlockGenerator(hidden_dim, hidden_dim),
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU(),
            self.final)

    def forward(self, x):
        return self.model(x)
    
class Model(nn.Module):
    
    def __init__(self, in_channels, hidden_dim = 64):
        super().__init__()
        
        self.conv_enc = ConvEnc(in_channels, hidden_dim = hidden_dim)
        self.conv_dec = ConvDec(5, hidden_dim = hidden_dim)
        enc_hparam = tx.modules.TransformerEncoder.default_hparams()
        enc_hparam['num_blocks']=3
        enc_hparam['dim']=hidden_dim
        enc_hparam['multihead_attention']['num_units']=hidden_dim
        enc_hparam['multihead_attention']['output_dim']=hidden_dim
        enc_hparam['poswise_feedforward']={'layers': [{'type': 'Linear', 'kwargs': {'in_features': hidden_dim, 'out_features': hidden_dim*2, 'bias': True}}, {'type': 'ReLU', 'kwargs': {'inplace': True}}, {'type': 'Dropout', 'kwargs': {'p': 0.1}}, {'type': 'Linear', 'kwargs': {'in_features': hidden_dim*2, 'out_features': hidden_dim, 'bias': True}}], 'name': 'ffn'}
        self.enc = tx.modules.TransformerEncoder(hparams = enc_hparam)
        self.enc.initialize_blocks()

    def forward(self, X):
        size = X.shape[2]
        c = self.conv_enc(X)
        shape = c.shape
        c = c.view(*c.shape[:2],-1).permute(2,0,1)
        e = self.enc(c, torch.full((c.shape[0], ), c.shape[1]))
        e = e.permute(1,2,0).reshape(shape)
        d = self.conv_dec(e[-1].unsqueeze(0))
        d = d[:,:,(d.shape[2]-size)//2:(d.shape[2]-size)//2+size,:]
        return d.squeeze()

In [7]:
model = Model(44).cuda()
wandb.watch(model)
learning_rate = 1e-4
opt = torch.optim.AdamW(model.parameters(), lr=learning_rate)
criterion = torch.nn.BCEWithLogitsLoss(pos_weight=torch.FloatTensor([1/0.0008]), reduction='mean', weight = dataset.mask.view(-1)).cuda()
num_steps = len(train_loader) * 10
print(num_steps)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=num_steps, eta_min=3e-5)
warmup_scheduler = warmup.UntunedLinearWarmup(opt)
class Weighted_MSE(nn.Module):
    
    def __init__(self, weight):
        super().__init__()
        self.weight = torch.nn.Parameter(weight.unsqueeze(0), requires_grad=False)

    def forward(self, X, target, mask):
        return torch.mean(self.weight * mask * (X - target) ** 2)

WMSE = Weighted_MSE(dataset.mask).cuda()

/home/holmes/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:70: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


20250


In [8]:
model.load_state_dict(torch.load("model_1.pt"))

<All keys matched successfully>

In [9]:
from sklearn.metrics import precision_recall_curve, roc_curve, confusion_matrix, explained_variance_score, mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
def subsample(A, B, length=100):
    assert(A.shape[0] == B.shape[0])
    l = A.shape[0]
    if l<length:
        return A,B
    L = np.ceil(l/length)*length
    d = int(L-l)
    A,B = np.pad(A, (int(d/2), d-int(d/2)),'edge'), np.pad(B, (int(d/2), d-int(d/2)),'edge')
    A,B = np.average(A.reshape(length, -1), axis=1), np.average(B.reshape(length, -1), axis=1)
    return A,B

def evaluate(loader, prefix="val"):
    model.eval()
    preds = []
    labels = []
    reg_y = []
    reg_Y = []
    with torch.no_grad():
        for indexes in tqdm(loader, desc="Evaluating..."):
            X, Y = dataset[indexes[0]]
            labels.append(Y[0].view(-1).data.numpy()[dataset.mask.view(-1).numpy()==1]/4)
            reg_Y.append(Y[1:].view(-1,4).data.numpy()[np.logical_and(dataset.mask.view(-1).numpy()==1, Y[0].view(-1).data.numpy()>0)].reshape(-1))
            X = X.cuda()
            pred = model(X)
            preds.append(torch.sigmoid(pred[0]).view(-1).data.cpu().numpy()[dataset.mask.view(-1).numpy()==1])
            reg_y.append((pred[1:]*8.92 + 405).view(-1,4).data.cpu().numpy()[np.logical_and(dataset.mask.view(-1).numpy()==1, Y[0].view(-1).data.numpy()>0)].reshape(-1))

    model.train()
    preds = np.concatenate(preds)
    reg_y = np.concatenate(reg_y)
    reg_Y = np.concatenate(reg_Y)
    preds_bin = preds>=0.5
#     print(Y[0])
    labels = np.concatenate(labels)>0
    fpr, tpr, threshold = roc_curve(labels, preds)
    fpr,tpr = subsample(fpr,tpr)
    fig_roc, ax = plt.subplots()
    ax.set_title('Receiver Operating Characteristic ({})'.format(prefix))
    ax.plot(fpr, tpr)
    ax.plot([0, 1], [0, 1], linestyle='--', label='Reference')
    ax.legend()
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')

    precision, recall, threshold = precision_recall_curve(labels, preds)
    precision, recall = subsample(precision, recall)
    fig_pr, ax = plt.subplots()
    ax.set_title('Precision Recall ({})'.format(prefix))
    ax.plot(recall, precision)
#     ax.plot([0, 1], [1, 0], linestyle='--', label='Reference')
    ax.set_xlabel('Recall')
    ax.set_ylabel('Precision')
    
    cf_matrix = confusion_matrix(labels,preds_bin)
    fig_confusion, ax = plt.subplots()
#     ax.set_title('Confusion Matrix ({})'.format(prefix))
    group_names = ['True Neg','False Pos','False Neg','True Pos']
    group_counts = ["{0:0.0f}".format(value) for value in
                    cf_matrix.flatten()]
    group_percentages = ["{0:.2%}".format(value) for value in
                         cf_matrix.flatten()/np.sum(cf_matrix)]
    lbl = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
              zip(group_names,group_counts,group_percentages)]
    lbl = np.asarray(lbl).reshape(2,2)
    sns.heatmap(cf_matrix, annot=lbl, fmt='', cmap='Blues', ax = ax)
    return {
        prefix+"_accuracy_score":accuracy_score(labels,preds_bin),
        prefix+"_balanced_accuracy_score":balanced_accuracy_score(labels,preds_bin), 
        prefix+"_f1_score":f1_score(labels, preds_bin), 
        prefix+"_precision_score":precision_score(labels, preds_bin), 
        prefix+"_recall_score":recall_score(labels, preds_bin), 
        prefix+"_roc_auc_score":roc_auc_score(labels, preds),
        prefix+"_pr" : fig_pr,
        prefix+"_roc" : fig_roc,
        prefix+"_confusion" : fig_confusion,
        prefix+"_MSE" : mean_squared_error(reg_Y, reg_y),
        prefix+"_MAE" : mean_absolute_error(reg_Y, reg_y),
        prefix+"_r2" : r2_score(reg_Y, reg_y),
        prefix+"_explained_variance_score" : explained_variance_score(reg_Y, reg_y),
    }

In [11]:
# results = evaluate(validation_loader)
# wandb.log(results)

In [12]:
c = 0

In [ ]:
model.train()
Eval = 500
for i in tqdm(range(100)):
    for indexes in tqdm(train_loader, desc="Training..."):
        if c % Eval == 0:
            results = evaluate(validation_loader)
            wandb.log(results)
        X, Y = dataset[indexes[0]]
        X, Y = X.cuda(), Y.cuda()
        pred = model(X)
        bceloss = criterion(pred[0].view(-1), Y[0].view(-1)/4)
        mseloss = WMSE(pred[1:], (Y[1:]-405)/8.92, (Y[0].unsqueeze(0)/4 + 0.001))
        loss = bceloss + 10 * mseloss
        opt.zero_grad()
        loss.backward()
        opt.step()
        lr_scheduler.step(lr_scheduler.last_epoch+1)
        warmup_scheduler.dampen()
        wandb.log({"loss": loss.item(),"MSE_loss": mseloss.item(),"BCE_loss": bceloss.item(), "lr": opt.param_groups[0]['lr']})
        c+=1
    torch.save(model.state_dict(), os.path.join(wandb.run.dir, 'model_{}.pt'.format(i)))
    torch.save(opt.state_dict(), os.path.join(wandb.run.dir, 'opt_{}.pt'.format(i)))
    results = evaluate(test_loader, prefix="test")
    wandb.log(results)

/opt/conda/conda-bld/pytorch_1591914855613/work/aten/src/ATen/native/TensorFactories.cpp:361: UserWarning: Deprecation warning: In a future PyTorch release torch.full will no longer return tensors of floating dtype by default. Instead, a bool fill_value will return a tensor of torch.bool dtype, and an integral fill_value will return a tensor of torch.long dtype. Set the optional `dtype` or `out` arguments to suppress this warning.


/home/holmes/anaconda3/lib/python3.7/site-packages/plotly/matplotlylib/renderer.py:410: UserWarning:

Bummer! Plotly can currently only draw Line2D objects from matplotlib that are in 'data' coordinates!

/home/holmes/anaconda3/lib/python3.7/site-packages/plotly/matplotlylib/renderer.py:512: UserWarning:

I found a path object that I don't think is part of a bar chart. Ignoring.

/home/holmes/anaconda3/lib/python3.7/site-packages/plotly/matplotlylib/renderer.py:474: UserWarning:

Dang! That path collection is out of this world. I totally don't know what to do with it yet! Plotly can only import path collections linked to 'data' coordinates

/home/holmes/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserWarning:

The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new 

/home/holmes/anaconda3/lib/python3.7/site-packages/plotly/matplotlylib/renderer.py:410: UserWarning:

Bummer! Plotly can currently only draw Line2D objects from matplotlib that are in 'data' coordinates!

/home/holmes/anaconda3/lib/python3.7/site-packages/plotly/matplotlylib/renderer.py:512: UserWarning:

I found a path object that I don't think is part of a bar chart. Ignoring.

/home/holmes/anaconda3/lib/python3.7/site-packages/plotly/matplotlylib/renderer.py:474: UserWarning:

Dang! That path collection is out of this world. I totally don't know what to do with it yet! Plotly can only import path collections linked to 'data' coordinates

/home/holmes/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserWarning:

The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new 

/home/holmes/anaconda3/lib/python3.7/site-packages/plotly/matplotlylib/renderer.py:410: UserWarning:

Bummer! Plotly can currently only draw Line2D objects from matplotlib that are in 'data' coordinates!

/home/holmes/anaconda3/lib/python3.7/site-packages/plotly/matplotlylib/renderer.py:512: UserWarning:

I found a path object that I don't think is part of a bar chart. Ignoring.

/home/holmes/anaconda3/lib/python3.7/site-packages/plotly/matplotlylib/renderer.py:474: UserWarning:

Dang! That path collection is out of this world. I totally don't know what to do with it yet! Plotly can only import path collections linked to 'data' coordinates

/home/holmes/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserWarning:

The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new 

/home/holmes/anaconda3/lib/python3.7/site-packages/plotly/matplotlylib/renderer.py:410: UserWarning:

Bummer! Plotly can currently only draw Line2D objects from matplotlib that are in 'data' coordinates!

/home/holmes/anaconda3/lib/python3.7/site-packages/plotly/matplotlylib/renderer.py:512: UserWarning:

I found a path object that I don't think is part of a bar chart. Ignoring.

/home/holmes/anaconda3/lib/python3.7/site-packages/plotly/matplotlylib/renderer.py:474: UserWarning:

Dang! That path collection is out of this world. I totally don't know what to do with it yet! Plotly can only import path collections linked to 'data' coordinates

/home/holmes/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserWarning:

The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new 

...

/home/holmes/anaconda3/lib/python3.7/site-packages/plotly/matplotlylib/renderer.py:410: UserWarning:

Bummer! Plotly can currently only draw Line2D objects from matplotlib that are in 'data' coordinates!

/home/holmes/anaconda3/lib/python3.7/site-packages/plotly/matplotlylib/renderer.py:512: UserWarning:

I found a path object that I don't think is part of a bar chart. Ignoring.

/home/holmes/anaconda3/lib/python3.7/site-packages/plotly/matplotlylib/renderer.py:474: UserWarning:

Dang! That path collection is out of this world. I totally don't know what to do with it yet! Plotly can only import path collections linked to 'data' coordinates

/home/holmes/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserWarning:

The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new 

/home/holmes/anaconda3/lib/python3.7/site-packages/plotly/matplotlylib/renderer.py:410: UserWarning:

Bummer! Plotly can currently only draw Line2D objects from matplotlib that are in 'data' coordinates!

/home/holmes/anaconda3/lib/python3.7/site-packages/plotly/matplotlylib/renderer.py:512: UserWarning:

I found a path object that I don't think is part of a bar chart. Ignoring.

/home/holmes/anaconda3/lib/python3.7/site-packages/plotly/matplotlylib/renderer.py:474: UserWarning:

Dang! That path collection is out of this world. I totally don't know what to do with it yet! Plotly can only import path collections linked to 'data' coordinates



/home/holmes/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserWarning:

The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.



/home/holmes/anaconda3/lib/python3.7/site-packages/plotly/matplotlylib/renderer.py:410: UserWarning:

Bummer! Plotly can currently only draw Line2D objects from matplotlib that are in 'data' coordinates!

/home/holmes/anaconda3/lib/python3.7/site-packages/plotly/matplotlylib/renderer.py:512: UserWarning:

I found a path object that I don't think is part of a bar chart. Ignoring.

/home/holmes/anaconda3/lib/python3.7/site-packages/plotly/matplotlylib/renderer.py:474: UserWarning:

Dang! That path collection is out of this world. I totally don't know what to do with it yet! Plotly can only import path collections linked to 'data' coordinates

/home/holmes/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserWarning:

The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new 

/home/holmes/anaconda3/lib/python3.7/site-packages/plotly/matplotlylib/renderer.py:410: UserWarning:

Bummer! Plotly can currently only draw Line2D objects from matplotlib that are in 'data' coordinates!

/home/holmes/anaconda3/lib/python3.7/site-packages/plotly/matplotlylib/renderer.py:512: UserWarning:

I found a path object that I don't think is part of a bar chart. Ignoring.

/home/holmes/anaconda3/lib/python3.7/site-packages/plotly/matplotlylib/renderer.py:474: UserWarning:

Dang! That path collection is out of this world. I totally don't know what to do with it yet! Plotly can only import path collections linked to 'data' coordinates

/home/holmes/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserWarning:

The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new 